# Settings

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
# sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..') # For Windows import
os.chdir('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..') # For Linux import
print(sep_local)
print(os.getcwd())

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='atari_pacman'

In [4]:
images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
#images_dir = 'C:\\projects\\pokemon\DS06\\'
validation_percentage = 25
valid_format = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [6]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

Using TensorFlow backend.


In [7]:
scale=1
image_size=(160//scale, 210//scale, 3)

batch_size = 32
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape= image_size
latents_dim = 30
intermediate_dim = 50

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [8]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode='episode_flat', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

In [9]:
import tensorflow as tf

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale = 1.0

In [13]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

1.0

In [14]:
inputs_shape

# Model's Layers definition

In [16]:
# units=100
# c=16
units=30
# c=(image_size[0]//4, image_size[1]//6, intermediate_dim//2)
c=(image_size[0]//4, image_size[1]//6, intermediate_dim//2)
menc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'), # units*9
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=units//5, kernel_size=3, strides=(2, 3), activation='relu'), #units*9
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product(c), activation=tf.nn.relu), # units=units*c*c
    tf.keras.layers.Reshape(target_shape=c), #target_shape=(c , c, units)
    tf.keras.layers.Conv2DTranspose(filters=units//5, kernel_size=3, strides=(2, 3), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=units, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]


120000

# Model definition

In [18]:
model_name = dataset_name+'Conv_GAVAE'
experiments_dir='experiments'+sep_local+model_name

In [19]:
from training.adversarial_basic.generative_adversarial.transformative.VAAE import VAAE as AE

In [20]:
inputs_shape=image_size

In [21]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    }

    ,
    
        {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [22]:
from utils.data_and_files.file_utils import create_if_not_exist

In [23]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [24]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonConvIVAAE\\var_save_dir'

In [25]:
#to restore trained model, set filepath=_restore

In [ ]:
from statistical.basic_adversarial_losses import \
    create_generative_discriminator_real_losses, \
    create_generative_discriminator_fake_losses, \
    create_generative_generator_fake_losses
    

generative_discriminator_losses = {
    'generative_discriminator_real_outputs': create_generative_discriminator_real_losses,
    'generative_discriminator_fake_outputs': create_generative_discriminator_fake_losses,
    'genertive_generator_fake_outputs': create_generative_generator_fake_losses,
}

In [26]:
ae = AE( 
    name= model_name,
    batch_size=batch_size, 
    episode_len= EPIS_LEN, 
    variables_params=variables_params,
    filepath=None
    )

Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (InputLa [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 32)      128       
_________________________________________________________________
flatten (Flatten)            (None, 320000)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                10240032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
activity_regularization (Act (None, 32)           

  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs (Input [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 320000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                10240032  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
activity_regularization_1 (A (None, 32

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 80000)             2640000   
_________________________________________________________________
reshape (Reshape)            (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 200, 200, 96)      27744     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 200, 200, 3)       2595      
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


xxx ['tf_op_layer_logpdf', 'tf_op_layer_z_latents', 'tf_op_layer_x_logits', 'inference_logvariance', 'inference_mean']


In [27]:
#ae.compile(metrics=None)
discr2gen_rate = 0.001
gen2trad_rate = 0.1

ae.compile(
    adversarial_losses=generative_discriminator_losses,
    adversarial_weights={'generator_weight': gen2trad_rate, 'discriminator_weight': discr2gen_rate}
)

Model: "pokemonConvIVAAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (InputLay [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)          (None, 32)           10240288    inference_logvariance_inputs[0][0]      
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 32)]         0           inference_logvariance[1][0]             
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_mul[0][0]            
_______________________________________________________

In [28]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonConvIVAAE\\csv_dir\\pokemonConvIVAAE.csv'

In [33]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

In [36]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)

Model: "generative_discriminator_real"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (InputLa [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 32)      128       
_________________________________________________________________
flatten (Flatten)            (None, 320000)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                10240032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
activity_regularization (Act (None,

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000002212E2CF108>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000022137FF1088>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000002212E2CF108>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000022137FF1088>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
Instructions for updating:
If using Keras pass *_constraint 

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000002212E2CF108>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000022160D21888>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000022137FF1088>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000002212E2CF108>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)